# Evaluation Retrieval System

In [7]:
import sys
import json
import random
from pathlib import Path


BASE_DIR = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent

if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

from vectordb.pinecone_client import PineconeVectorDB
from vectordb.metrics.eval_metric import calculate_hit_rate, calculate_mrr, calculate_precision_at_k

In [ ]:
PINECONE_API_KEY = "your api key"
BASE_URL = "https://fahmiaziz-api-embedding.hf.space/api/v1"
SAMPLE_DATA = "data/pubmed_data.json"

### Utility Function

In [3]:
def load_sample_data(path: str):
    with open(path, "r") as f:
        data = json.load(f)
    
    return data

### Load Data

In [4]:
data = load_sample_data(SAMPLE_DATA)
for i, item in enumerate(data[:3]):
    print(f"Sample {i+1}:")
    print(f"ID: {item['id']}")
    print(f"Question: {item['question'][:100]}...")
    print(f"Context: {item['context'][:150]}...")
    print(f"\n{'-'*60}\n")

Sample 1:
ID: 21645374
Question: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?...
Context: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in ...

------------------------------------------------------------

Sample 2:
ID: 16418930
Question: Landolt C and snellen e acuity: differences in strabismus amblyopia?...
Context: Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical...

------------------------------------------------------------

Sample 3:
ID: 9488747
Question: Syncope during bathing in infants, a pediatric form of water-induced urticaria?...
Context: Apparent life-threatening events in infants are a difficult and frequent problem in pediatric practice. The prognosis is uncertain because of risk of ...

-------------------------------------

## VectorDB

### Initialize VectorDB

In [5]:
vector_db = PineconeVectorDB(
    api_key=PINECONE_API_KEY,
    embedding_api_url=BASE_URL
)
vector_db.delete_index("hybrid")

vector_db.create_index_db("hybrid", 1024)

2025-11-07 06:54:10.265 | SUCCESS  | vectordb.pinecone_client:delete_index:265 - 🗑️  Deleted index: hybrid
2025-11-07 06:54:10.878 | INFO     | vectordb.pinecone_client:create_index_db:46 - 📦 Creating index: hybrid
2025-11-07 06:54:21.528 | SUCCESS  | vectordb.pinecone_client:create_index_db:58 - ✅ Index hybrid created successfully
2025-11-07 06:54:23.781 | INFO     | vectordb.pinecone_client:create_index_db:64 - 📊 Index stats: {'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


### Push Data to Index

In [6]:
# push data
sample_data = random.sample(data, 50)

await vector_db.push_data_to_index(
    documents=sample_data,
    index_name="hybrid",
    batch_size=10
)

2025-11-07 06:55:48.305 | INFO     | vectordb.pinecone_client:push_data_to_index:81 - 📤 Uploading 50 documents in batches of 10...
2025-11-07 06:55:48.311 | DEBUG    | vectordb.pinecone_client:push_data_to_index:90 - 
🔄 Processing batch 1/5 (10 docs)...
2025-11-07 06:56:24.693 | INFO     | vectordb.pinecone_client:_push_hybrid_batch:121 -   ✓ Generated 10 dense embeddings (dim: 1024)
2025-11-07 06:56:27.045 | INFO     | vectordb.pinecone_client:_push_hybrid_batch:126 -   ✓ Generated 10 sparse embeddings
2025-11-07 06:56:32.601 | INFO     | vectordb.pinecone_client:_push_hybrid_batch:148 -   ✓ Uploaded 10 hybrid vectors
2025-11-07 06:56:32.608 | INFO     | vectordb.pinecone_client:push_data_to_index:99 - ✅ Batch 1/5 uploaded
2025-11-07 06:56:32.612 | DEBUG    | vectordb.pinecone_client:push_data_to_index:90 - 
🔄 Processing batch 2/5 (10 docs)...
2025-11-07 06:56:58.035 | INFO     | vectordb.pinecone_client:_push_hybrid_batch:121 -   ✓ Generated 10 dense embeddings (dim: 1024)
2025-11-07

In [9]:
# lets try some query
idx = 6

res = await vector_db.query_with_rerank(
    query=sample_data[idx]["question"],
    index_name="hybrid",
    alpha=1.0,
    initial_top_k=10,
    final_top_k=5
)

2025-11-07 07:03:02.756 | INFO     | vectordb.pinecone_client:query_with_rerank:232 - 🎯 Reranking top 10 results to 5...
2025-11-07 07:03:15.191 | SUCCESS  | vectordb.pinecone_client:query_with_rerank:253 - ✅ Reranking complete!


In [16]:
print(f"Original ID: {sample_data[idx]['id']}")
print(f"Question: {sample_data[idx]['question']}\n")

for item in res:
    print(f"Context: {item['context'][:100]}")
    print(f"ID Document: {item['id']}")
    print(f"Rerank Score: {item['rerank_score']}")
    print("====="*10)

Original ID: 12690589
Question: Can continuous physical training counteract aging effect on myoelectric fatigue?

Context: To compare the myoelectric onset of muscle fatigue in physically active trained young skiers with re
ID Document: 12690589
Rerank Score: 1.0
Context: To explore the secondary benefits of treadmill training for people in the chronic stage of recovery 
ID Document: 18955431
Rerank Score: 0.0006737615913152695
Context: The aim of this study was to investigate the role of menopausal transition and menopausal symptoms i
ID Document: 17429333
Rerank Score: 0.00016124015382956713
Context: It is unclear whether intravenous glycoprotein IIb/IIIa inhibitors or ischemic time might modify any
ID Document: 26175531
Rerank Score: 7.94369861978339e-06
Context: This was a study to compare the results of mitral valve (MV) repair and MV replacement for the treat
ID Document: 22440363
Rerank Score: 0.0


## Evaluate 

In [17]:
async def evaluate_query(
    query: str,
    ground_truth_id: str,
    index_name: str,
    alpha: float = 1.0,
    top_k: int = 5,
    ) :
    results = await vector_db.query_with_rerank(
        query=query,
        index_name=index_name,
        alpha=alpha,
        initial_top_k=top_k * 2,  
        final_top_k=top_k
    )
    retrieved_ids = [r['id'] for r in results]

        # Calculate metrics
    hit = calculate_hit_rate(retrieved_ids, ground_truth_id)
    mrr = calculate_mrr(retrieved_ids, ground_truth_id)
    precision = calculate_precision_at_k(retrieved_ids, [ground_truth_id], k=top_k)

    result = {
        'hit': hit,
        'mrr': mrr,
        'precision': precision,
    }
    return result

In [18]:
num_samples = 25
random_indices = random.sample(range(len(sample_data)), min(num_samples, len(sample_data)))

total_hit_rate = 0
total_mrr = 0
total_precision = 0
evaluated_count = 0

print(f"Evaluating {len(random_indices)} random samples...")

for idx in random_indices:
    print(f"\nProcessing sample {idx} (ID: {sample_data[idx]['id']})...")
    try:
        result = await evaluate_query(
            query=sample_data[idx]["question"],
            ground_truth_id=sample_data[idx]["id"],
            index_name="hybrid",
            alpha=1.0,
            top_k=5
        )
        total_hit_rate += result['hit']
        total_mrr += result['mrr']
        total_precision += result['precision']
        evaluated_count += 1
        print(f"Metrics for sample {idx}: Hit Rate={result['hit']}, MRR={result['mrr']:.4f}, Precision@5={result['precision']:.4f}")

    except Exception as e:
        print(f"⚠️  Error evaluating sample {idx}: {e}")


print("\n===================================")
print(f"Average Evaluation Results (over {evaluated_count} samples):")
print("===================================")

if evaluated_count > 0:
    avg_hit_rate = total_hit_rate / evaluated_count
    avg_mrr = total_mrr / evaluated_count
    avg_precision = total_precision / evaluated_count

    print(f"Average Hit Rate: {avg_hit_rate:.4f}")
    print(f"Average MRR: {avg_mrr:.4f}")
    print(f"Average Precision@{5}: {avg_precision:.4f}")
else:
    print("No samples were successfully evaluated.")

Evaluating 25 random samples...

Processing sample 25 (ID: 16564683)...


2025-11-07 07:10:38.254 | INFO     | vectordb.pinecone_client:query_with_rerank:232 - 🎯 Reranking top 10 results to 5...
2025-11-07 07:10:56.591 | SUCCESS  | vectordb.pinecone_client:query_with_rerank:253 - ✅ Reranking complete!


Metrics for sample 25: Hit Rate=1, MRR=1.0000, Precision@5=0.2000

Processing sample 12 (ID: 18537964)...


2025-11-07 07:11:02.531 | INFO     | vectordb.pinecone_client:query_with_rerank:232 - 🎯 Reranking top 10 results to 5...
2025-11-07 07:11:17.690 | SUCCESS  | vectordb.pinecone_client:query_with_rerank:253 - ✅ Reranking complete!


Metrics for sample 12: Hit Rate=1, MRR=1.0000, Precision@5=0.2000

Processing sample 33 (ID: 26578404)...


...

Metrics for sample 10: Hit Rate=1, MRR=1.0000, Precision@5=0.2000

Processing sample 42 (ID: 25957366)...


2025-11-07 07:19:55.100 | INFO     | vectordb.pinecone_client:query_with_rerank:232 - 🎯 Reranking top 10 results to 5...
2025-11-07 07:20:09.026 | SUCCESS  | vectordb.pinecone_client:query_with_rerank:253 - ✅ Reranking complete!


Metrics for sample 42: Hit Rate=1, MRR=1.0000, Precision@5=0.2000

Processing sample 8 (ID: 24013712)...


2025-11-07 07:20:14.775 | INFO     | vectordb.pinecone_client:query_with_rerank:232 - 🎯 Reranking top 10 results to 5...
2025-11-07 07:20:29.507 | SUCCESS  | vectordb.pinecone_client:query_with_rerank:253 - ✅ Reranking complete!


Metrics for sample 8: Hit Rate=1, MRR=1.0000, Precision@5=0.2000

Average Evaluation Results (over 25 samples):
Average Hit Rate: 1.0000
Average MRR: 1.0000
Average Precision@5: 0.2000
